In [1]:
pip install pandas sqlalchemy pyodbc openpyxl

Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
from sqlalchemy import create_engine
import urllib

# 📍 متغيرات الاتصال الخاصة بك
SERVER_NAME = 'SHAMS\SQLEXPRESS'
DATABASE_NAME = 'UK_train'

# 🛠️ إنشاء سلسلة اتصال لـ Windows Authentication (Trusted_Connection=yes)
# ملاحظة: يجب أن يكون 'ODBC Driver 17 for SQL Server' مثبتاً.
params = urllib.parse.quote_plus(
    f'DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={SERVER_NAME};DATABASE={DATABASE_NAME};Trusted_Connection=yes;'
)

# 🔗 بناء سلسلة الاتصال النهائية لـ SQLAlchemy
connection_string = f'mssql+pyodbc:///?odbc_connect={params}'

# ⚙️ إنشاء محرك الاتصال (Engine)
try:
    engine = create_engine(connection_string)
    
    # 🧪 اختبار الاتصال (اختياري)
    with engine.connect() as connection:
        print("✅ تم الاتصال بقاعدة البيانات بنجاح!")
        
except Exception as e:
    print(f"❌ خطأ في الاتصال بقاعدة البيانات: {e}")
    # إذا ظهر خطأ، قد يكون بسبب عدم تثبيت ODBC Driver أو خطأ في اسم الخادم.

# الآن، يمكنك استخدام هذا الـ `engine` في مرحلة التحميل (المرحلة الرابعة)
# باستخدام df.to_sql(..., con=engine, ...)

<>:6: SyntaxWarning: invalid escape sequence '\S'
<>:6: SyntaxWarning: invalid escape sequence '\S'
C:\Users\shams\AppData\Local\Temp\ipykernel_16024\960324907.py:6: SyntaxWarning: invalid escape sequence '\S'
  SERVER_NAME = 'SHAMS\SQLEXPRESS'


✅ تم الاتصال بقاعدة البيانات بنجاح!


In [9]:
# ⚠️ تم تحديث المسار واسم الملف بناءً على مدخلاتك
EXCEL_FILE_PATH = "C:/Users/shams/Desktop/DEPI project/Data Modeling.xlsx"

# 📍 المرحلة الثانية: قراءة واستخلاص البيانات (Extraction)
print("\n--- 📄 بدء قراءة ملف Excel ---")

try:
    dataframes = pd.read_excel(EXCEL_FILE_PATH, sheet_name=None)
    
    # 💡 أسماء الشيتات التي تم اكتشافها:
    sheet_names = list(dataframes.keys())
    
    print(f"☑️ تم قراءة {len(dataframes)} جدول/شيت من الملف بنجاح. الأسماء المُكتشفة:")
    print(f"   {sheet_names}")
    
except FileNotFoundError:
    print(f"❌ خطأ: لم يتم العثور على الملف في المسار: {EXCEL_FILE_PATH}. يرجى التحقق من المسار.")
    exit()

# 📍 المرحلة الثالثة: التنظيف والتحويل (Transformation - تنظيف الأعمدة)
print("\n--- 📝 بدء تنظيف أسماء الأعمدة (مع معالجة الخطأ) ---")

for sheet_name, df in dataframes.items():
    
    # 🌟 الخطوة الجديدة: تحويل جميع أسماء الأعمدة إلى نوع String (str)
    # هذا يحل مشكلة AttributeError إذا كانت الأعمدة أرقاماً أو تواريخ
    df.columns = df.columns.astype(str)
    
    # تنظيف أسماء الأعمدة (إزالة المسافات والرموز)
    df.columns = df.columns.str.replace(' ', '_', regex=False).str.replace(r'[^\w]+', '', regex=True)
    
    print(f"   - تم تنظيف أعمدة شيت: {sheet_name}")

print("☑️ انتهى التنظيف. الجداول جاهزة الآن للتحميل.")


--- 📄 بدء قراءة ملف Excel ---
☑️ تم قراءة 8 جدول/شيت من الملف بنجاح. الأسماء المُكتشفة:
   ['Sheet1', 'Fact_TrainRides', 'Dim_Route', 'Dim_Ticket', 'Dim_Purchase', 'Dim_Status', 'Dim_Date', 'Dim_Time']

--- 📝 بدء تنظيف أسماء الأعمدة (مع معالجة الخطأ) ---
   - تم تنظيف أعمدة شيت: Sheet1
   - تم تنظيف أعمدة شيت: Fact_TrainRides
   - تم تنظيف أعمدة شيت: Dim_Route
   - تم تنظيف أعمدة شيت: Dim_Ticket
   - تم تنظيف أعمدة شيت: Dim_Purchase
   - تم تنظيف أعمدة شيت: Dim_Status
   - تم تنظيف أعمدة شيت: Dim_Date
   - تم تنظيف أعمدة شيت: Dim_Time
☑️ انتهى التنظيف. الجداول جاهزة الآن للتحميل.


In [11]:
# 💡 الترتيب الحاسم! هذا يضمن عدم حدوث خطأ المفاتيح الأجنبية.
LOAD_ORDER = [
    'Dim_Date',
    'Dim_Time',
    'Dim_Status',
    'Dim_Purchase',
    'Dim_Ticket',
    'Dim_Route',
    'Fact_TrainRides' 
]

print("\n--- 🚀 بدء إدخال البيانات إلى SQL Server ---")
# 

# تأكد أن المتغير 'engine' لا يزال معرفاً من كود الاتصال
# وكذلك المتغير 'dataframes' من كود القراءة والتنظيف

for table_name in LOAD_ORDER:
    if table_name not in dataframes:
        # هذه الحالة لن تحدث طالما البيانات صحيحة، ولكن للتأكد
        print(f"⚠️ تحذير: اسم الشيت {table_name} غير موجود في الـ DataFrames.")
        continue
        
    df_to_load = dataframes[table_name]
    
    try:
        # استخدام دالة to_sql لإرسال DataFrame إلى قاعدة البيانات
        # name=table_name: اسم الجدول الذي سينشأ في SQL Server
        # con=engine: محرك الاتصال الذي أنشأناه
        # if_exists='replace': ينشئ الجدول من جديد (مناسب للتحميل الكامل)
        df_to_load.to_sql(
            name=table_name,
            con=engine,
            if_exists='replace', 
            index=False,         
            chunksize=5000       
        )
        print(f"✅ تم تحميل جدول: {table_name}")
        
    except Exception as e:
        # إذا حدث خطأ أثناء التحميل، سيتم طباعته هنا
        print(f"❌ فشل تحميل جدول {table_name}. الخطأ: {e}")
        # يمكنك إيقاف البرنامج هنا إذا كنت تريد مراجعة الخطأ قبل المتابعة.
        
# معالجة الشيت الإضافي (Sheet1)
if 'Sheet1' in dataframes:
    print(f"\n⚠️ تم تجاهل الشيت: Sheet1 (غير جزء من نموذج البيانات).")

print("\n🎉 اكتملت عملية التحميل! يرجى مراجعة قاعدة البيانات 'UK_train'.")


--- 🚀 بدء إدخال البيانات إلى SQL Server ---
✅ تم تحميل جدول: Dim_Date
✅ تم تحميل جدول: Dim_Time
✅ تم تحميل جدول: Dim_Status
✅ تم تحميل جدول: Dim_Purchase
✅ تم تحميل جدول: Dim_Ticket
✅ تم تحميل جدول: Dim_Route
✅ تم تحميل جدول: Fact_TrainRides

⚠️ تم تجاهل الشيت: Sheet1 (غير جزء من نموذج البيانات).

🎉 اكتملت عملية التحميل! يرجى مراجعة قاعدة البيانات 'UK_train'.
